In [ ]:
!pip install pandas==1.5.3
!pip install sentence-transformers accelerate -U
!pip install transformers[torch] -U
# !pip install accelerate -U --force-reinstall

import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1
Accelerate version: 0.27.2
Transformers version: 4.38.2


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd

file_folder = '/content/drive/My Drive/224_project/'
# Specify Google Drive folder ID and file names
folder_id = "1FwqVD1UvjaoSC8UksUBhVkPzPQy95iTg"
train_file_name = 'sustainability_business_activities_training.csv'
test_file_name = 'sustainability_business_activities_test.csv'
EPA_file_name = 'EPA_EmissionData.csv'

# Define the fields to potentially introduce errors
fields = ['Business Activity Description', 'Business Activity Vendor', 'Business Activity Comment']
num_max_error_columns = 1
naics_title_column_name = '2017 NAICS Title'
activity_cost_column_name = 'Business Activity Cost USD'
epa_emission_factor_column_name = 'Supply Chain Emission Factors without Margins'
separator_string = " [SEP] "
error_column_append_text = " Error"
combined_text_column_name = 'combined_text'
encoded_label_column_name = 'encoded_labels'

demo_mode = True

def find_file_id_by_name(drive, folder_id, file_name):
    """Search for a file by name in the specified Google Drive folder."""
    query = f"'{folder_id}' in parents and trashed=false and title='{file_name}'"
    file_list = drive.ListFile({'q': query}).GetList()
    return file_list[0]['id'] if file_list else None

epa_file_id = find_file_id_by_name(drive, folder_id, EPA_file_name)
epa_data = drive.CreateFile({'id': epa_file_id})
epa_data.GetContentFile(EPA_file_name)
factor_df = pd.read_csv(EPA_file_name)
print(f"{len(factor_df)} of rows in factor csv")
# print(factor_df.head())

train_file_id = find_file_id_by_name(drive, folder_id, train_file_name)
activity_data = drive.CreateFile({'id': train_file_id})
activity_data.GetContentFile(train_file_name)
activity_df = pd.read_csv(train_file_name)
print(f"{len(activity_df)} of rows in training csv")
# print(activity_df.head())

# Load the test data
test_file_id = find_file_id_by_name(drive, folder_id, test_file_name)
activity_test_data = drive.CreateFile({'id': test_file_id})
activity_test_data.GetContentFile(test_file_name)
test_df = pd.read_csv(test_file_name)
print(f"{len(test_df)} of rows in test csv")
# print(test_df.head())

1016 of rows in factor csv
13744 of rows in training csv
2756 of rows in test csv


In [ ]:
# import torch

In [ ]:
# device = torch.device('cpu')

In [ ]:
label_dict = {value: idx for idx, value in enumerate(activity_df[naics_title_column_name].unique())}
activity_df[encoded_label_column_name] = activity_df[naics_title_column_name].map(label_dict)


In [ ]:
import pandas as pd
import numpy as np
import random

# Helper functions
def introduce_minor_errors(text):
    """Introduce minor spelling mistakes in the text."""
    errors_introduced = 0
    max_errors = random.randint(2, 3)  # Decide to introduce 2 or 3 minor errors

    while errors_introduced < max_errors and len(text) > 4:  # Ensure text is long enough to alter
        error_type = random.choice(['substitute', 'omit', 'swap'])
        error_index = random.randint(1, len(text) - 2)  # Avoid beginning and end of the text for simplicity

        if error_type == 'substitute':
            # Substitute a character with a nearby character (mimicking common typing errors)
            substitutions = {'a': 's', 's': 'a', 'd': 'f', 'i': 'o', 'o': 'p', 'e': 'r', 'r': 't'}
            if text[error_index] in substitutions:
                text = text[:error_index] + substitutions[text[error_index]] + text[error_index + 1:]
                errors_introduced += 1

        elif error_type == 'omit':
            # Omit a character
            text = text[:error_index] + text[error_index + 1:]
            errors_introduced += 1

        elif error_type == 'swap':
            # Swap two adjacent characters
            if error_index < len(text) - 1:  # Ensure there's a character to swap with
                text = text[:error_index] + text[error_index + 1] + text[error_index] + text[error_index + 2:]
                errors_introduced += 1
    return text

def introduce_major_errors(text):
    """Replace or scramble parts of the text to introduce major errors."""
    # Randomly choose between scrambling or inserting irrelevant text
    if random.random() < 0.5:
        return ''.join(random.sample(text, len(text)))
    else:
        return "Irrelevant text " + ''.join(random.sample(text, len(text)))
    return text

# Function to randomly apply either minor or major errors to a text
def apply_random_error(text):
    if random.random() < 0.20:  # 15% chance to introduce an error
        #error_type = random.choice(['minor', 'major'])
        #if error_type == 'minor':
        #    return introduce_minor_errors(text)
        #else:
            return introduce_major_errors(text)   # only major errors
    return text


def apply_errors_with_limit(row, fields, max_errors=num_max_error_columns):
    """
    Randomly apply errors to a limited number of fields in a row.

    Parameters:
    - row: The DataFrame row to apply errors to.
    - fields: A list of field names to potentially apply errors to.
    - max_errors: Maximum number of fields to apply errors to.
    """
    # Randomly decide how many fields to apply errors to (0 to max_errors)
    errors_to_apply = random.randint(0, max_errors)

    # Randomly select the fields where errors will be applied
    fields_with_errors = random.sample(fields, errors_to_apply)

    # Apply errors to the selected fields
    for field in fields:
        if field in fields_with_errors:
            row[field + error_column_append_text] = apply_random_error(row[field])
        else:
            row[field + error_column_append_text] = row[field]

    return row

def combine_text_fields(row, fields):
    """
    Combine multiple text fields into a single combined text string.

    Parameters:
    - row: A DataFrame row containing the text fields.
    - fields: A list of field names to be combined.

    Returns:
    - combined_text: A string containing the combined text from the specified fields.
    """
    combined_parts = []
    for field in fields:
        # Assuming the 'Error' versions of fields are already in the DataFrame
        error_field_name = f"{field}{error_column_append_text}"
        if error_field_name in row:
            field_label = field.replace(" ", "_")  # Replace spaces with underscores for label
            combined_parts.append(f"{field_label}: {row[error_field_name]}")
    combined_text = separator_string.join(combined_parts)
    return combined_text

In [ ]:
from datetime import datetime

current_date = datetime.now()
# Format month as 3-letter abbreviation and day as a number
formatted_date = current_date.strftime('%b%d')

# Apply errors to 2 or fewer fields for each row
activity_df = activity_df.apply(lambda row: apply_errors_with_limit(row, fields), axis=1)
# Apply the function to each row of the DataFrame to create the 'combined_text' column
activity_df[combined_text_column_name] = activity_df.apply(lambda row: combine_text_fields(row, fields), axis=1)
# Now, 'combined_text' contains the concatenated texts with either minor or major errors introduced
temp_train_file_name = f'train_{formatted_date}.csv'
activity_df.to_csv(temp_train_file_name)

# Apply errors to 2 or fewer fields for each row
test_df = test_df.apply(lambda row: apply_errors_with_limit(row, fields), axis=1)
# Combine the possibly altered text fields into a new 'combined_text' column
test_df[combined_text_column_name] = test_df.apply(lambda row: combine_text_fields(row, fields), axis=1)
temp_test_file_name = f'test_{formatted_date}.csv'
test_df.to_csv(temp_test_file_name)


In [ ]:
import numpy as np
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Split data into features and labels
# activity_df['combined_text'] = activity_df['Business Activity Description'] + " " + activity_df['Vendor'] + " " + activity_df['Comment']
X = activity_df[combined_text_column_name]  # Feature
y = activity_df[encoded_label_column_name]  # Assuming 'label' is already encoded as numeric labels

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize and fit the TF-IDF vectorizer on training data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint

# Define the parameter grid
param_dist = {
    "n_estimators": [600, 800, 1000, 1200, 1400],  # Increased range based on previous best
    "max_depth": [50, 60, 80],  # Explore deeper trees and some constraints
    "min_samples_split": [8, 10, 12, 15, 18],  # Narrowing down around the best found value
    "min_samples_leaf": [1, 2, 3],  # Exploring around the best found value
    "max_features": ['sqrt', 'log2', None]  # Adding exploration around 'log2'
}

if demo_mode:
  print("In the demo mode, not running the random_search....")
else:
  # Initialize the RandomForest model
  clf = RandomForestClassifier(random_state=42)

  # Initialize RandomizedSearchCV
  n_iter_search = 20
  random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search, cv=5, scoring='accuracy')

  # Fit the model
  random_search.fit(X_train_tfidf, y_train)

  print("Best parameters:", random_search.best_params_)
  # Best parameters: {'n_estimators': 800, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 60}
  # Best parameters: {'n_estimators': 1200, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 60}
  # Best parameters: {'n_estimators': 1000, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 80}


In the demo mode, not running the random_search....


In [ ]:
# Previously selected best parameter sets
selected_params = [
    {'n_estimators': 800, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 60},
    {'n_estimators': 1200, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 60},
    {'n_estimators': 1000, 'min_samples_split': 15, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 80}
]

# Choose a parameter set (e.g., the first one)
chosen_params = selected_params[0]
if demo_mode:
  print("In demo mode, moving forward with params: {chosen_params}")
else:
  chosen_params = random_search.best_params_
# Initialize the RandomForest model with the chosen parameters
best_clf = RandomForestClassifier(**chosen_params, random_state=42)
# Fit the model
best_clf.fit(X_train_tfidf, y_train)
# Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)
# Make predictions on the test data
y_pred = best_clf.predict(X_test_tfidf)

# Convert y_test to a DataFrame for easier manipulation
y_test_df = pd.DataFrame(y_test)
y_test_df.reset_index(drop=True, inplace=True)

# Convert y_pred to a DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=['Predicted'])

# Concatenate the true labels, predicted labels, and the original text features for comparison
results_df = pd.concat([y_test_df, y_pred_df, X_test.reset_index(drop=True)], axis=1)
results_df.columns = ['True_Label', 'Predicted_Label', 'Features']

# Filter to find where predictions were incorrect
incorrect_predictions = results_df[results_df['True_Label'] != results_df['Predicted_Label']]

'''
# Add a prefix to 'Features' to indicate these are from true labels
incorrect_predictions['Features_True_Label'] = 'True label: ' + incorrect_predictions['Features']
# Since 'Features' based on 'Predicted_Label' doesn't exist directly, we replicate 'Features' column
# and add a prefix to indicate these are hypothetically what you'd expect for predicted labels
# Note: This does not fetch different 'Features' text for 'Predicted_Label'; it's illustrative
incorrect_predictions['Features_Predicted_Label'] = 'Predicted label: ' + incorrect_predictions['Features']
'''

# Set option to display full text without truncation for investigation
pd.set_option('display.max_colwidth', None)
print(f"Rows {len(incorrect_predictions)} in Incorrect Predictions: ")
print(incorrect_predictions[['True_Label', 'Predicted_Label', 'Features']])
# Reset display option
pd.reset_option('display.max_colwidth')

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In demo mode, moving forward with params: {chosen_params}
Rows 8 in Incorrect Predictions: 
      True_Label  Predicted_Label  \
550            5                7   
698            3                7   
1135           3                7   
1157           3                7   
1168           6                7   
1172           8                7   
1207           9                0   
1927           3                7   

                                                                                                                                                                                                                              Features  
550                       Business_Activity_Description: Irrelevant text wrl Rea sfgong reoeienayeenwtrenge bmae [SEP] Business_Activity_Vendor: RenewFlow Tech [SEP] Business_Activity_Comment: Contributing to sustainability goals.  
698             Business_Activity_Description: tdnm optTncn lsoiedgrssor i foriea ra [SEP] Business_Activity

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer  # Assuming you've already fitted this with your training data

# Assuming clf is your trained RandomForestClassifier and tfidf_vectorizer is your fitted TF-IDF vectorizer

test_df[encoded_label_column_name] = test_df[naics_title_column_name].apply(lambda x: label_dict.get(x, -1))  # Unseen labels get -1

X_test_new = test_df[combined_text_column_name]  # Feature
y_test_new = test_df[encoded_label_column_name]  # Replace 'label' with the actual column name for labels in your test data

# Transform the test data using the already fitted TF-IDF vectorizer
X_test_new_tfidf = tfidf_vectorizer.transform(X_test_new)

# Make predictions on the new test data
y_pred_new = best_clf.predict(X_test_new_tfidf)

# Evaluate the model on the new test data
print("New Test Data - Classification Report:\n", classification_report(y_test_new, y_pred_new))
print("New Test Data - Accuracy:", accuracy_score(y_test_new, y_pred_new))


New Test Data - Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       257
           1       1.00      1.00      1.00       230
           2       1.00      1.00      1.00       237
           3       1.00      0.99      1.00       252
           4       1.00      1.00      1.00       250
           5       1.00      1.00      1.00       254
           6       1.00      0.99      1.00       242
           7       0.98      1.00      0.99       253
           8       1.00      1.00      1.00       257
           9       1.00      1.00      1.00       253
          10       1.00      1.00      1.00       271

    accuracy                           1.00      2756
   macro avg       1.00      1.00      1.00      2756
weighted avg       1.00      1.00      1.00      2756

New Test Data - Accuracy: 0.997822931785196


In [ ]:
# Convert y_pred_new and y_test_new to a DataFrame for easier manipulation
results_df = pd.DataFrame({combined_text_column_name: X_test_new, 'Actual Label': y_test_new, 'Predicted Label': y_pred_new})

# Filter the DataFrame to only include rows where the prediction failed
failed_predictions = results_df[results_df['Actual Label'] != results_df['Predicted Label']]

# Display the details of rows with failed predictions
print("Rows with Failed Predictions:")
pd.set_option('display.max_colwidth', None)  # For pandas versions < 1.0, use -1 instead of None
print(failed_predictions)
pd.reset_option('display.max_colwidth')

Rows with Failed Predictions:
                                                                                                                                                                                                                        combined_text  \
240                              Business_Activity_Description: Baola ienesllmn tsu ndnt qgeeairwp [SEP] Business_Activity_Vendor: AlleyEquip Sales [SEP] Business_Activity_Comment: Providing expert advice for equipment selection.   
1033                Business_Activity_Description: gnnntststEnilialidxida  ahe iob [SEP] Business_Activity_Vendor: ExhibitPast Creators [SEP] Business_Activity_Comment: Incorporating multimedia elements for dynamic presentations.   
1070            Business_Activity_Description: hn aneoenallBel dwegica ncg oltnytmonyai [SEP] Business_Activity_Vendor: FrameFix Services [SEP] Business_Activity_Comment: Ensuring optimal lane conditions with regular maintenance.   
1216                   Business_Activi

In [ ]:
# Ensure that '2017 NAICS Title' in both dataframes are of the same format for accurate mapping
# Map 'Supply Chain Emission Factors without Margins' from factor_df to test_df based on '2017 NAICS Title'
test_df['Emission Factor'] = test_df[naics_title_column_name].map(
    factor_df.set_index(naics_title_column_name)[epa_emission_factor_column_name]
)

# Calculate the emissions for each row before grouping
test_df['Calculated_Emissions'] = test_df['Emission Factor'] * test_df[activity_cost_column_name]

# Selecting unique '2017 NAICS Title' and their corresponding 'Emission Factor' to avoid duplicates
unique_emission_factors = test_df[[naics_title_column_name, 'Emission Factor']].drop_duplicates()

# Step 1: Match NAICS Titles in test_df with those in factor_df
# This step is simplified due to direct matching by '2017 NAICS Title'.
# In real-world scenarios, consider complexities of matching titles.

# Group 'test_df' by '2017 NAICS Title' and sum the 'Calculated_Emissions' for each group, also count the occurrences
aggregated_emissions = test_df.groupby(naics_title_column_name).agg(
    Total_Emissions=('Calculated_Emissions', 'sum'),
    Count=('Calculated_Emissions', 'count')
).reset_index()

# Print the total emissions and count for each title
for index, row in aggregated_emissions.iterrows():
    print(f"Title: {row[naics_title_column_name]}, Total Emissions: {row['Total_Emissions']:.2f}, Count: {row['Count']}")


# Calculate and print overall totals using DataFrame functions
total_count = aggregated_emissions['Count'].sum()
total_emissions = aggregated_emissions['Total_Emissions'].sum()

print(f"\nTotal Count of All Rows: {total_count}")
print(f"Total of Total Emissions: {total_emissions:.2f}")

Title: Automobile Driving Schools, Total Emissions: 15331.12, Count: 250
Title: Bowling Centers, Total Emissions: 41183.58, Count: 242
Title: Historical Sites, Total Emissions: 19081.32, Count: 253
Title: Kidney Dialysis Centers, Total Emissions: 14414.15, Count: 257
Title: New Single-Family Housing Construction (except For-Sale Builders), Total Emissions: 22194.55, Count: 271
Title: Offices of Dentists, Total Emissions: 9963.62, Count: 230
Title: Political Organizations, Total Emissions: 16379.77, Count: 257
Title: Sewage Treatment Facilities, Total Emissions: 80527.14, Count: 254
Title: Support Activities for Metal Mining, Total Emissions: 68293.40, Count: 253
Title: Timber Tract Operations, Total Emissions: 27541.68, Count: 237
Title: Uranium-Radium-Vanadium Ore Mining, Total Emissions: 137873.47, Count: 252

Total Count of All Rows: 2756
Total of Total Emissions: 452783.82
